# **Esercitazione su image classification**
Nell'esercitazione odierna utilizzeremo le *Convolutional Neural Network* (CNN) per applicazioni di riconoscimento di volti (*Face Recognition*). 

Faremo uso del framework **TensorFlow**, sfruttando la libreria open-source **Keras** appositamente progettata per permettere una rapida prototipazione di reti neurali profonde.

Alcuni link di approfondimento:
- Introduzione a TensorFlow con utile schema grafico delle [API disponibili](https://ekababisong.org/gcp-ml-seminar/tensorflow/#navigating-through-the-tensorflow-api)
- [Keras](https://keras.io/)

Nello specifico potranno essere utilizzate due reti (VGG-16 e ResNet-50) pre-addestrate sui dataset [VGGFace](http://www.robots.ox.ac.uk/~vgg/data/vgg_face/) (contenente oltre 2 milioni di immagini di volti appartenenti a più di 2000 soggetti) e [VGGFace2](http://www.robots.ox.ac.uk/~vgg/data/vgg_face2/) (contenente oltre 3 milioni di immagini di volti appartenenti a più di 9000 soggetti).

L'obiettivo dell'esercitazione è quello di utilizzare una CNN pre-addestrata come *feature extractor* per il riconoscimento di volti.

# **Operazioni preliminari**
Prima di incominciare, è necessario eseguire alcune operazioni preliminari.

Per ovviare a problemi di retrocompatibilità, il codice seguente aggiorna la versione di alcune librerie necessarie allo svolgimento dell'esercitazione. 


In [ ]:
%tensorflow_version 1.x

!pip uninstall -y h5py
!pip install h5py==2.10.0

Eseguendo la cella sottostante tutto il materiale necessario per lo svolgimento dell'esercitazione verrà scaricato sulla macchina remota. Alla fine dell'esecuzione selezionare il tab **Files** per verificare che tutto sia stato scaricato correttamente.

In [ ]:
!wget http://bias.csr.unibo.it/VR/Esercitazioni/DBs/ImageClassification/FaceScrubSubset_Celebrities.zip
!wget http://bias.csr.unibo.it/VR/Esercitazioni/PythonUtilities.zip

!unzip /content/FaceScrubSubset_Celebrities.zip
!unzip /content/PythonUtilities.zip

!rm /content/FaceScrubSubset_Celebrities.zip
!rm /content/PythonUtilities.zip

## **Installazione della libreria Keras-vggface**
Per poter caricare velocemente i modelli preaddestrati sul dataset VGGFace2 si utilizzeranno alcune funzionalità della libreria [**Keras-vggface**](https://github.com/rcmalli/keras-vggface).

Eseguire la cella sottostante per installare la libreria.

In [ ]:
!pip install git+https://github.com/rcmalli/keras-vggface.git
!pip install keras_applications

# **Import delle librerie**
Per prima cosa è necessario eseguire l'import delle librerie utilizzate durante l'esecitazione.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
import math
from tensorflow import keras
from keras.models import Model
from keras_vggface.vggface import VGGFace

import vr_utilities

# **Dataset**
Il dataset ultilizzato è composto da immagini RGB di volti di persone famose. In particolare utilizzeremo un sottoinsieme del [FaceScrub](http://vintage.winklerbros.net/facescrub.html) contenente 1590 immagini di 530 soggetti diversi (3  immagini per ciascuno di essi, 2 per il training e 1 per il test).

Visto il numero esiguo di immagini (1060 per il dataset di training), cercare di addestrare da zero una CNN complessa (partendo da pesi random) risulta impossibile.

In [ ]:
db_path = '/content/Celebrities'
train_filelist = 'TrainingSet.txt'
test_filelist = 'TestSet.txt'
labelnames_list = 'LabelNames.txt'

print('Caricamento in corso ...')
or_train_x, train_y = vr_utilities.load_labeled_dataset(train_filelist, db_path)
or_test_x, test_y = vr_utilities.load_labeled_dataset(test_filelist, db_path)

label_names = vr_utilities.load_label_names(labelnames_list, db_path)

print('Shape training set:', or_train_x.shape)
print('Shape test set:', or_test_x.shape)

La cella seguente contiene il codice per mostrare alcune immagini del training set. Guardando alcuni esempi si può facilmente notare la grande variabilità in termini di:
- posa;
- illuminazione;
- espressione.

In [ ]:
rows = 3
columns = 6

plt.rcParams.update({'font.size': 20})
_, axs = plt.subplots(rows, columns, squeeze=False,figsize=(30, 15))
samples = random.sample(range(len(label_names)), columns)

for j in range(columns):
    idx=samples[j]
    sel_train_images=[or_train_x[k] for k in np.where(train_y==idx)[0]]
    sel_test_images=[or_test_x[k] for k in np.where(test_y==idx)[0]]
    sel_images=sel_train_images+sel_test_images
    axs[0, j].set_title(label_names[idx])
    for i in range(rows):
        axs[i, j].axis('off')
        axs[i, j].imshow(sel_images[i])

# **Creazione del modello**
Vediamo ora come creare un modello con il supporto di Keras e della libreria Keras-vggface. Utilizzare la variabile *model_name* per selezionare quale modello utilizza tra i due disponibili (VGG-16 e ResNet-50).

In [ ]:
model_name='vgg16'
#model_name='resnet50'

# create a vggface2 model
model = VGGFace(model=model_name)

print('Inputs: %s' % model.inputs)
print('Outputs: %s' % model.outputs)

## **Visualizzazione del modello**
Eseguendo la cella seguente è possibile stampare un riepilogo testuale della struttura della rete.

In [ ]:
model.summary()

Se si preferisce una visualizzazione grafica, eseguire la cella seguente.

In [ ]:
keras.utils.plot_model(model,show_shapes=True, show_layer_names=True)

# **Creazione dell'estrattore di feature**
L'addestramento di una CNN su un nuovo problema, richiede un
training set etichettato di notevoli dimensioni .

In alternativa al training da zero, possiamo utilizzare una rete esistente (pre-trained) per estrarre le feature generate ai livelli intermedi durante il passo forward ([*Transfer Learning*](https://cs231n.github.io/transfer-learning/)) e utilizzare queste feature per:
1. addestrare un classificatore esterno (es. SVM) a
riconoscere i pattern del nuovo dominio applicativo;
2. stimare il grado di similarità tra feature estratte da immagini differenti utilizzando una metrica (es. distanza coseno).

L'operazione di estrazione delle feature consiste nel calcolare, per ogni immagine fornita in input, l'output della rete al livello desiderato (*layer_name*).

Per evitare, durante il passo *forward*, di attraversare livelli non necessari della rete si può creare una nuova istanza della classe [**Model**](https://keras.io/api/models/model/) il cui input sara il medesimo del modello originale mentre l'ouput sarà rappresentato dal livello da cui si vogliono estrarre le feature (*layer_name*).

In [ ]:
if (model_name=='vgg16'):
  layer_name = 'fc7' #VGG-16
elif (model_name=='resnet50'):
  layer_name = 'avg_pool' #ResNet-50

feature_extractor = Model(inputs=model.input,outputs=model.get_layer(layer_name).output)

print('Inputs: %s' % feature_extractor.inputs)
print('Outputs: %s' % feature_extractor.outputs)

## **Visualizzazione del nuovo modello**
Eseguendo la cella seguente è possibile stampare un riepilogo testuale della struttura della rete da utilizzare come feature extractor.

In [ ]:
feature_extractor.summary()

Se si preferisce una visualizzazione grafica, eseguire la cella seguente.

In [ ]:
keras.utils.plot_model(feature_extractor,show_shapes=True, show_layer_names=True)

# ***Pre-processing* delle immagini**
I modelli utilizzati sono stati addestrati con delle immagini pre-elaborate. Sarà necessario eseguire le medesime operazioni sia sul training che sul test set prima di poterli utilizzare.

## **Mapping**
Se le immagini presentano delle intensità nel range [0;1] (come nel nostro caso), per prima cosa si dovrà "mappare" le intensità nel range [0;255]. Si esegua la cella seguente per effettuare il *mapping*. 

In [ ]:
print('Range originale: [',np.min(or_train_x),';',np.max(or_train_x),']')

norm_train_x=or_train_x*255
norm_test_x=or_test_x*255

print('Range ri-mappato: [',np.min(norm_train_x),';',np.max(norm_train_x),']')

## **Normalizzazione**
Per rendere i modelli robusti rispetto a variazioni del contrasto e della luminosità, le immagini utilizzate per l'addestramento sono state preventivamente normalizzate (singolarmente) sottraendo a ogni pixel l'intensità media dell'intero dataset di training.

Si esegua la cella seguente per normalizzare tutte le immagini sottraendovi l'intensità media del rispettivo training set.

In [ ]:
if (model_name=='vgg16'):
  mean_value=np.array([129.1863,104.7624,93.5940]) #RGB
elif (model_name=='resnet50'):
  mean_value=np.array([131.0912,103.8827,91.4953]) #RGB
  
print('Normalizzazione in corso ...')
norm_train_x = norm_train_x-mean_value
norm_test_x = norm_test_x-mean_value
print('Normalizzazione completata')

## **Conversione RGB->BGR**
Per ragioni "storiche", la maggior parte delle reti è stata addestrata con immagini il cui ordine dei canali è BGR e non RGB come ci si potrebbe aspettare. Pertanto, sarà prima necessario invertire l'ordine dei canali delle nostre immagini.

Si esegua la cella successiva per invertire l'ordine dei canali da RGB a BGR.

In [ ]:
norm_train_x = norm_train_x[..., ::-1]
norm_test_x = norm_test_x[..., ::-1]

# **Estrazione delle feature**
Per estrarre le feature è sufficiente richiamare il metodo [**predict(...)**](https://keras.io/api/models/model_training_apis/#predict-method) del nostro estrattore (*feature_extractor*).

Eseguire la cella seguente per estrarre le feature dal training e dal test set.

In [ ]:
print('Estrazione delle feature...')
train_features_x=feature_extractor.predict(norm_train_x)
test_features_x=feature_extractor.predict(norm_test_x)

print('Shape ndarray delle feature di train: ', train_features_x.shape)
print('Shape ndarray delle feature di test: ', test_features_x.shape)

Per comodità, può essere utile rimuovere le dimensioni unitarie tramite la funzione [**squeeze(...)**](https://numpy.org/doc/stable/reference/generated/numpy.squeeze.html) di NumPy.

In [ ]:
train_features_x=np.squeeze(train_features_x)
test_features_x=np.squeeze(test_features_x)

print('Shape ndarray delle feature di train: ', train_features_x.shape)
print('Shape ndarray delle feature di test: ', test_features_x.shape)

# **Face Recognition**
Le feature appena estratte possono essere direttamente utilizzate insieme alla [distanza coseno](https://en.wikipedia.org/wiki/Cosine_similarity) per effettuare *face recognition* sulle nostre immagini.

Dati due vettori **A** e **B**, la distanza coseno può essere calcolata come: 

\begin{align}
D_C(\mathbf{A},\mathbf{B})=1-\frac{\mathbf{A} \cdot{} \mathbf{B}}{\lVert \mathbf{A} \rVert \lVert \mathbf{B} \rVert}
\end{align}

La funzione **compute_cosine_distances(...)**, definita nella cella seguente, calcola le distanze coseno delle feature di una immagine di test (*query_features_x*) da tutte le feature del training set (*train_features_x*). Questa implementazione permette di calcolare la norma di ogni immagine di test una sola volta. 

In [ ]:
def compute_cosine_distances(train_features_x,query_features_x):
  cosine_distances=[]
  norm_query=np.linalg.norm(query_features_x)
  for train_feature in train_features_x:
    norm_train=np.linalg.norm(train_feature)
    cos_dist=1-np.dot(query_features_x, train_feature)/(norm_query*norm_train)
    cosine_distances.append(cos_dist)

  return np.asarray(cosine_distances)

## **Test**
La cella sottostante calcola tutte le distanze coseno tra le feature del dataset di test e quelle di training memorizzandole nella variabile *test_distances*.

In [ ]:
test_distances=[]
print('Calcolo distanze coseno ...')
for test_features in test_features_x:
  test_distances.append(compute_cosine_distances(train_features_x,test_features))

test_distances=np.asarray(test_distances)

print('Shape ndarray delle distanze: ', test_distances.shape)

È possibile misurare l'accuratezza del sistema di *face recognition* implementato eseguendo la cella successiva.

In [ ]:
test_distances_sorted_indices=np.argsort(test_distances,axis=1)

predicted_y=train_y[test_distances_sorted_indices[:,0]]

errors = predicted_y != test_y

accuracy=1-(errors.sum()/len(errors))
print('Accuracy sul test set: %.3f' % (accuracy))

## **Visualizzazione errori**
La cella seguente permette di visualizzare le immagini di test che vengono classificate in maniera errata. Sopra ad ogni immagine è riportato il nome del soggetto mentre a lato le classi più probabili.

In [ ]:
error_indices = np.where(errors == True)[0]

if error_indices.shape[0] > 0:
  # Visualizzazione immagini
  image_per_row = 2
  top_class_count = 5

  row_count=math.ceil(len(error_indices)/image_per_row)
  column_count=image_per_row
  plt.rcParams.update({'font.size': 12})
  _, axs = plt.subplots(row_count, column_count,figsize=(20, 4*row_count),squeeze=False)
    
  for i in range(row_count):
    for j in range(column_count):
      axs[i,j].axis('off')

  for i in range(len(error_indices)):
    q = i // image_per_row
    r = i % image_per_row
    idx = error_indices[i]
    
    axs[q,r].imshow(or_test_x[idx])
    axs[q,r].set_title(label_names[test_y[idx]])

    best_indices=test_distances_sorted_indices[idx,0:2*top_class_count]
    best_distances=test_distances[idx,best_indices]
    
    best_y = train_y[best_indices]
    _, unique_indices = np.unique(best_y, return_index=True)
    unique_indices=np.sort(unique_indices)
    
    text=''
    for j in range(top_class_count):
        text+='{}: {:.3f}\n'.format(label_names[best_y[unique_indices[j]]],best_distances[unique_indices[j]])
    
    axs[q,r].text(330, 150, text, horizontalalignment='left', verticalalignment='center')

# **Esercizio**
Utilizzare il sistema implementato per verificare a quale tra le celebrità presenti nel dataset assomigliate maggiormente.

A tal fine:

1. scattare una foto con il proprio volto in primo piano;
2. ritagliarla per ottenere un'immagine quadrata (rapporto 1:1);
3. riscalare l'immagine a una dimensione 224 x 224 pixel; 
4. trasferire l'immagine ottenuta su **Colab** utilizzando la funzione *Upload* del tab **Files**;
5. caricare l'immagine in una variabile (per farlo può essere utile la funzione [**imread(...)**](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.imread.html) della libreria [**Matplotlib**](https://matplotlib.org/));
6. effettuare il *pre-processing* dell'immagine;
7. estrarre le feature utilizzando l'estrattore creato;
8. calcolare la distanza coseno tra le feature estratte e quelle del training set; 
9. visualizzare, utilizzando la libreria Matplotlib, il nome e le foto delle 3 celebrità più somiglianti.

In [ ]:
# Soluzione:
image_file_path = '/content/MatteoFerrara.png'
similar_count = 3

# Punto 5
my_image = plt.imread(image_file_path)

# Punto 6
my_norm_image = my_image*255
my_norm_image = my_norm_image-mean_value
my_norm_image = my_norm_image[..., ::-1]

# Punto 7
my_image_features=feature_extractor.predict(np.expand_dims(my_norm_image, axis=0))
my_image_features=np.squeeze(my_image_features)

# Punto 8
my_distances=compute_cosine_distances(train_features_x,my_image_features)
my_distances_sorted_indices=np.argsort(my_distances)
best_indices=my_distances_sorted_indices[0:2*similar_count] # scegli 6, almeno 3 classi ...
best_distances=my_distances[best_indices]

# Punto 9
plt.imshow((my_image))
plt.axis('off')

best_y = train_y[best_indices]
_, unique_indices = np.unique(best_y, return_index=True)
unique_indices=np.sort(unique_indices)
unique_labels = best_y[unique_indices]

text=''
for j in range(similar_count):
    text+='{}: {:.3f}\n'.format(label_names[best_y[unique_indices[j]]],best_distances[unique_indices[j]])

plt.text(my_image.shape[1]*1.05, my_image.shape[0]*0.5, text, horizontalalignment='left', verticalalignment='center')

similar_face_indices=[i for i in best_indices if train_y[i] in unique_labels[:similar_count]]

plt.rcParams.update({'font.size': 13})
_, axs = plt.subplots(1, len(similar_face_indices),figsize=(20, 4))
    
for i in range(len(similar_face_indices)):
    idx=similar_face_indices[i]
    axs[i].axis('off')
    axs[i].imshow(or_train_x[idx])
    axs[i].set_title(label_names[train_y[idx]])